In [1]:
# ---------------------------------------------------------
#   Python equivalent for data prep + hurdle NB models
#   (approximation of glmmTMB functionality)
# ---------------------------------------------------------

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
#lower part goes here

# ---------------------------------------------------------
# Load data
# ---------------------------------------------------------

df = pd.read_csv("EctosQuantitativeFinal.csv", na_values="NA")

# ---------------------------------------------------------
# Convert categorical variables
# ---------------------------------------------------------

categorical_vars = [
    "Number", "Condition"
]

for var in categorical_vars:
    df[var] = df[var].astype("category")

# ---------------------------------------------------------
# Scale numeric variables
# ---------------------------------------------------------

scaler = StandardScaler()
scale_vars = [
    "Ticks", "Weight",
    "Recaptures", "Fleas"
]

for var in scale_vars:
    df[var + "z"] = scaler.fit_transform(df[[var]])

# ---------------------------------------------------------
# HURDLE NEGATIVE BINOMIAL MODEL (Python approximation)
# (No random effects; NOT truncated NB like glmmTMB)
# ---------------------------------------------------------

formula_main = """
Recaptures ~ Ticks + Weight + Fleas + Condition + Number
"""

hurdle_model = smf.poisson_hurdle(
    formula=formula_main,
    data=df,
    dist="negbin"
)

result_main = hurdle_model.fit()
print("\n=========== MAIN HURDLE MODEL ===========")
print(result_main.summary())


ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [ ]:
# ---------------------------------------------------------
# Python version of the Distance Moved model workflow
# ---------------------------------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from patsy import dmatrix

# ---------------------------------------------------------
# Load data
# ---------------------------------------------------------

OF_DATA = pd.read_csv("EctosQuantitativeFinal.csv")

# Convert categorical variables
categorical_vars = ["Number", "Species", "Tail", "Sex", "Condition", "Weight", "Length",
                    "Fleas", "Ticks", "Recaptures"]

for var in categorical_vars:
    OF_DATA[var] = OF_DATA[var].astype("category")

# ---------------------------------------------------------
# Fit linear mixed model (equivalent to lmer)
# ---------------------------------------------------------

#YOUVE MADE IT HERE SO FAR
formula = """
DISTz ~ DENSyc + MM2 + GRID + SEQ_OF + TRAPz + TIMEz +
         SEXc + AGEc + REPROc + MASSz + DENSdz +
         FLEAc + TICKc + BOTFLYc + MITEc + FLEAc:TICKc
"""

m_DIST = smf.mixedlm(
    formula=formula,
    data=OF_DATA,
    groups=OF_DATA["ID"],
    re_formula="~1"
).fit()

print("\n==================== LINEAR MIXED MODEL ====================")
print(m_DIST.summary())

# ---------------------------------------------------------
# "rptGaussian" substitute:
# Extract variance explained by random effects
# ---------------------------------------------------------

var_RE = m_DIST.random_effects
var_components = m_DIST.cov_re
var_resid = m_DIST.scale

print("\n==================== Repeatability Approximation ====================")
print("Random effect variance:", var_components)
print("Residual variance:", var_resid)

# ---------------------------------------------------------
# EFFECT PLOT for FLEAc:TICKc interaction
# (similar to effect('FLEAc:TICKc') in R)
# ---------------------------------------------------------

# Make prediction grid
pred_df = OF_DATA.copy()
pred_df["FLEAc"] = pred_df["FLEAc"].cat.codes
pred_df["TICKc"] = pred_df["TICKc"].cat.codes

# Unique combinations
newdat = (
    pred_df.groupby(["FLEAc", "TICKc"])
    [["FLEAc", "TICKc"]]
    .first()
    .reset_index()
)

# Predict
newdat["fit"] = m_DIST.predict(newdat)

# Approximate 95% CI using model standard error
# (R's effect() produces similar intervals)
se = m_DIST.bse.mean()
newdat["lower"] = newdat["fit"] - 1.96 * se
newdat["upper"] = newdat["fit"] + 1.96 * se

print("\n==================== Interaction Predictions ====================")
print(newdat)

# ---------------------------------------------------------
# Plot equivalent to your R script
# ---------------------------------------------------------

plt.figure(figsize=(6, 5))

# Blue = Ticks absent (TICKc = 0)
subset0 = newdat[newdat["TICKc"] == 0]
plt.plot(subset0["FLEAc"], subset0["fit"], color="blue", lw=2)
plt.fill_between(subset0["FLEAc"], subset0["lower"], subset0["upper"],
                 color="blue", alpha=0.15)

# Red = Ticks present (TICKc = 1)
subset1 = newdat[newdat["TICKc"] == 1]
plt.plot(subset1["FLEAc"], subset1["fit"], color="red", lw=2)
plt.fill_between(subset1["FLEAc"], subset1["lower"], subset1["upper"],
                 color="red", alpha=0.15)

plt.xlabel("FLEA presence (0=no, 1=yes)")
plt.ylabel("Distance moved (z)")
plt.title("FLEA × TICK Interaction Effect on Distance Moved")
plt.legend(["Ticks absent", "Ticks present"])

plt.show()

# ---------------------------------------------------------
# Extract coefficients table (equivalent to cbind)
# ---------------------------------------------------------

DIST_coeff = pd.DataFrame({
    "Estimate": m_DIST.params,
    "SE": m_DIST.bse,
    "Lower 95%": m_DIST.conf_int()[0],
    "Upper 95%": m_DIST.conf_int()[1]
})

print("\n==================== COEFFICIENTS TABLE ====================")
print(DIST_coeff)
